import libs and read the data

In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings(action='once')

In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',20)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df     = pd.read_csv('/content/drive/MyDrive/kaggle_readmission_comp/train.csv')
df_test= pd.read_csv('/content/drive/MyDrive/kaggle_readmission_comp/test.csv')

split features and labels

In [6]:
df_features = df.iloc[:,:-1]
df_labels = df.iloc[:,-1]

In [7]:
from sklearn.preprocessing import LabelEncoder
labels_encoder = LabelEncoder()
labels = labels_encoder.fit_transform(df_labels)
labels = pd.Series(labels,name='readmitted')

In [8]:
labels.value_counts()

2    38405
1    24881
0     7950
Name: readmitted, dtype: int64

the weight feature

In [9]:
#0:50 / 50:125 / >125
for i in range(len(df_features)):
  if df_features['weight'][i] == '?':
    None
  elif df_features['weight'][i] in ['[0-25)' , '[25-50)']:
    df_features['weight'][i] = '[0-50)'
  elif df_features['weight'][i] in ('[50-75)' , '[75-100)' , '[100-125)'):
    df_features['weight'][i] = '[50-125)'
  else :
    df_features['weight'][i] = '>125'
                                    
for i in range(len(df_test)):
  if df_test['weight'][i] == '?':
    None
  elif df_test['weight'][i] in ['[0-25)' , '[25-50)']:
    df_test['weight'][i] = '[0-50)'
  elif df_test['weight'][i] in ('[50-75)' , '[75-100)' , '[100-125)'):
    df_test['weight'][i] = '[50-125)'
  else :
    df_test['weight'][i] = '>125'

<ipython-input-9-e7340a33b57d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['weight'][i] = '[50-125)'


In [10]:
df_weight = pd.get_dummies(df_features['weight'])
df_weight.head()

,>125,?,[0-50),[50-125)
0,0,1,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0


In [11]:
df_features[df_weight.columns] = df_weight

In [12]:
df_features.drop(columns =['weight','?'],inplace=True)

In [13]:
df_weight = pd.get_dummies(df_test['weight'])
df_test[df_weight.columns] = df_weight
df_test.drop(columns=['weight','?'],inplace=True)
df_test.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,>125,[0-50),[50-125)
0,109962834,23779836,AfricanAmerican,Female,[60-70),2,22,1,4,MC,InternalMedicine,46,1,26,0,0,0,250.7,707,785,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0
1,240829854,32387580,Caucasian,Male,[80-90),2,13,1,3,MC,?,42,0,12,0,0,0,285,585,242,9,None,None,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,0,0,0
2,44942994,19436418,?,Female,[20-30),2,18,4,4,?,Family/GeneralPractice,71,3,20,0,0,0,486,428,518,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,0,0
3,302469824,142233539,AfricanAmerican,Female,[60-70),1,22,7,7,MD,?,79,1,13,0,0,0,434,584,585,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0
4,172075704,79818876,Caucasian,Male,[60-70),2,22,1,7,?,Surgery-Vascular,33,2,37,0,0,4,440,496,287,9,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,0,0,0


payer code
40% of people who are labeled(<30) have no payer code

In [14]:
df['payer_code'].value_counts()

?     28218
MC    22647
HM     4380
SP     3522
BC     3302
MD     2469
CP     1765
UN     1729
CM     1328
OG      720
PO      410
DM      388
CH      101
WC       97
OT       63
MP       54
SI       42
FR        1
Name: payer_code, dtype: int64

In [15]:
#? /MC & HM / others
for i in range(len(df_features)):
  if df_features['payer_code'][i] not in ['?','MC','HM','SP','BC','MD','CP','UN','CM']:
    df_features['payer_code'][i] ='other'
for i in range(len(df_test)):
  if df_test['payer_code'][i] not in ['?','MC','HM','SP','BC','MD','CP','UN','CM']:
    df_test['payer_code'][i] ='other'

<ipython-input-15-8b57190f5d90>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['payer_code'][i] ='other'


In [16]:
df_payer_code = pd.get_dummies(df_features['payer_code'])
df_features[df_payer_code.columns] = df_payer_code
df_features.drop(columns=['?','payer_code'],inplace=True)
df_features.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other
0,197920182,110382426,Caucasian,Female,[80-90),3,1,7,5,Emergency/Trauma,40,2,25,0,0,0,682,276,403,9,None,>7,No,No,No,No,No,No,Steady,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,0,0,0,0,0,0,0,1,0,0,0,0
1,30870324,1276911,Caucasian,Male,[70-80),3,5,1,3,Orthopedics,22,1,22,0,0,0,715,496,414,8,None,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,0,0,0,0
2,126616944,92115693,Caucasian,Female,[70-80),3,3,1,8,Family/GeneralPractice,33,5,5,0,0,0,157,197,576,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,0,0,0,0,0,0,0,1,0,0,0,0
3,303646118,32010471,Caucasian,Male,[50-60),6,1,7,7,?,52,1,15,0,0,4,276,403,585,6,>200,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,0,0,0,0,0,0,0,0,0,0,0
4,109444026,2402271,Caucasian,Male,[40-50),1,1,6,2,InternalMedicine,47,2,10,0,0,1,564,276,250,5,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,0,0,1,0


In [17]:
df_payer_test = pd.get_dummies(df_test['payer_code'])
df_test[df_payer_test.columns] = df_payer_test
df_test.drop(columns=['?','payer_code'],inplace=True)
df_test.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other
0,109962834,23779836,AfricanAmerican,Female,[60-70),2,22,1,4,InternalMedicine,46,1,26,0,0,0,250.7,707,785,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,1,0,0,0,0
1,240829854,32387580,Caucasian,Male,[80-90),2,13,1,3,?,42,0,12,0,0,0,285,585,242,9,None,None,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,0,0,0,0,0,0,0,1,0,0,0,0
2,44942994,19436418,?,Female,[20-30),2,18,4,4,Family/GeneralPractice,71,3,20,0,0,0,486,428,518,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,0,0,0,0,0,0,0,0,0,0,0
3,302469824,142233539,AfricanAmerican,Female,[60-70),1,22,7,7,?,79,1,13,0,0,0,434,584,585,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,1,0,0,0
4,172075704,79818876,Caucasian,Male,[60-70),2,22,1,7,Surgery-Vascular,33,2,37,0,0,4,440,496,287,9,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
pd.concat([df_payer_code,labels],axis=1).corr().tail()

,?,BC,CM,CP,HM,MC,MD,SP,UN,other,readmitted
MD,-0.153465,-0.041775,-0.026116,-0.030202,-0.048500,-0.129362,1.000000,-0.043214,-0.029885,-0.031163,-0.002755
SP,-0.184711,-0.050281,-0.031433,-0.036352,-0.058374,-0.155701,-0.043214,1.000000,-0.035970,-0.037507,-0.000732
UN,-0.127738,-0.034772,-0.021738,-0.025139,-0.040369,-0.107676,-0.029885,-0.035970,1.000000,-0.025939,0.026512
other,-0.133199,-0.036258,-0.022667,-0.026214,-0.042095,-0.112279,-0.031163,-0.037507,-0.025939,1.000000,0.009479
readmitted,-0.001881,0.038487,0.007927,0.022367,0.000893,-0.036229,-0.002755,-0.000732,0.026512,0.009479,1.000000


preparing the features

In [19]:
df_features.drop(columns =['encounter_id','medical_specialty'],inplace=True)
df_test.drop(columns =['encounter_id','medical_specialty'],inplace=True)

patient nbr feature

In [20]:
df_features['patient_nbr'] = df_features['patient_nbr'].apply(lambda x:df_features['patient_nbr'].value_counts()[x])

In [21]:
df_test['patient_nbr'] = df_test['patient_nbr'].apply(lambda x:df_test['patient_nbr'].value_counts()[x])

admission , discharge and time in hogpital feature

In [22]:
df_features['patient_sum'] = np.zeros(len(df_features))
for i in range(len(df_features)):
  df_features['patient_sum'].iloc[i] =1.5*df_features[
      'admission_type_id'][i] +df_features[
      'admission_source_id'][i]+1.5*df_features[
      'time_in_hospital'][i] + df_features['discharge_disposition_id'][i]

/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [23]:
df_test['patient_sum'] = np.zeros(len(df_test))
for i in range(len(df_test)):
  df_test['patient_sum'].iloc[i] =1.5*df_test[
      'admission_type_id'][i] +df_test[
      'admission_source_id'][i]+1.5*df_test[
      'time_in_hospital'][i] + df_test['discharge_disposition_id'][i]

In [24]:
pd.concat([df_features['patient_sum'],df_features['admission_type_id'],df_features['admission_source_id']
           ,df_features['discharge_disposition_id'],df_features['time_in_hospital'],labels],axis = 1).corr()

,patient_sum,admission_type_id,admission_source_id,discharge_disposition_id,time_in_hospital,readmitted
patient_sum,1.000000,0.329671,0.487237,0.694029,0.584907,-0.047179
admission_type_id,0.329671,1.000000,0.109405,0.078278,-0.015621,0.010672
admission_source_id,0.487237,0.109405,1.000000,0.018381,-0.008497,-0.028770
discharge_disposition_id,0.694029,0.078278,0.018381,1.000000,0.160108,-0.012002
time_in_hospital,0.584907,-0.015621,-0.008497,0.160108,1.000000,-0.059465
readmitted,-0.047179,0.010672,-0.028770,-0.012002,-0.059465,1.000000


the age feature

In [25]:
df_features['age'].value_counts()

[70-80)     18277
[60-70)     15662
[50-60)     12125
[80-90)     12048
[40-50)      6763
[30-40)      2659
[90-100)     1941
[20-30)      1165
[10-20)       479
[0-10)        117
Name: age, dtype: int64

In [26]:
df_age = pd.get_dummies(df_features['age'])
df_features[df_age.columns] = df_age
df_features.drop(columns=['age'],inplace=True)
df_features.head()

,patient_nbr,race,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other,patient_sum,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100)
0,3,Caucasian,Female,3,1,7,5,40,2,25,0,0,0,682,276,403,9,None,>7,No,No,No,No,No,No,Steady,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,0,0,0,0,0,0,0,1,0,0,0,0,20.0,0,0,0,0,0,0,0,0,1,0
1,1,Caucasian,Male,3,5,1,3,22,1,22,0,0,0,715,496,414,8,None,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,0,0,0,0,15.0,0,0,0,0,0,0,0,1,0,0
2,1,Caucasian,Female,3,3,1,8,33,5,5,0,0,0,157,197,576,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,0,0,0,0,0,0,0,1,0,0,0,0,20.5,0,0,0,0,0,0,0,1,0,0
3,1,Caucasian,Male,6,1,7,7,52,1,15,0,0,4,276,403,585,6,>200,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,0,0,0,0,0,0,0,0,0,0,0,27.5,0,0,0,0,0,1,0,0,0,0
4,2,Caucasian,Male,1,1,6,2,47,2,10,0,0,1,564,276,250,5,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,0,0,1,0,11.5,0,0,0,0,1,0,0,0,0,0


In [27]:
pd.concat([df_age,labels],axis=1).corr().tail()

,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100),readmitted
[60-70),-0.021532,-0.043679,-0.068451,-0.104534,-0.171937,-0.240433,1.000000,-0.311868,-0.239513,-0.088848,-0.000538
[70-80),-0.023828,-0.048335,-0.075749,-0.115678,-0.190267,-0.266066,-0.311868,1.000000,-0.265047,-0.098321,-0.022405
[80-90),-0.018300,-0.037121,-0.058175,-0.088840,-0.146124,-0.204337,-0.239513,-0.265047,1.000000,-0.075510,-0.022926
[90-100),-0.006788,-0.013770,-0.021580,-0.032956,-0.054205,-0.075800,-0.088848,-0.098321,-0.075510,1.000000,0.012501
readmitted,0.020261,0.015372,-0.002923,0.013556,0.008937,0.025734,-0.000538,-0.022405,-0.022926,0.012501,1.000000


In [28]:
df_age = pd.get_dummies(df_test['age'])
df_test[df_age.columns] = df_age
df_test.drop(columns=['age'],inplace=True)
df_test.head()

,patient_nbr,race,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other,patient_sum,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100)
0,1,AfricanAmerican,Female,2,22,1,4,46,1,26,0,0,0,250.7,707,785,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,1,0,0,0,0,32.0,0,0,0,0,0,0,1,0,0,0
1,1,Caucasian,Male,2,13,1,3,42,0,12,0,0,0,285,585,242,9,None,None,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,0,0,0,0,0,0,0,1,0,0,0,0,21.5,0,0,0,0,0,0,0,0,1,0
2,1,?,Female,2,18,4,4,71,3,20,0,0,0,486,428,518,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,0,0,0,0,0,0,0,0,0,0,0,31.0,0,0,1,0,0,0,0,0,0,0
3,1,AfricanAmerican,Female,1,22,7,7,79,1,13,0,0,0,434,584,585,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,1,0,0,0,41.0,0,0,0,0,0,0,1,0,0,0
4,2,Caucasian,Male,2,22,1,7,33,2,37,0,0,4,440,496,287,9,None,None,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,0,0,0,0,0,0,0,0,0,0,0,0,36.5,0,0,0,0,0,0,1,0,0,0


the diag features

In [29]:
def group_ICD9(df, diag):
  for index, val in enumerate(df[diag]):
      if val == '?':
          df[diag].iloc[index] = '?'

      elif val.startswith('V') or val.startswith('E'):
          df[diag].iloc[index] = 'E_or_V'

      else:
      
        if int(val.split('.')[0]) > 1 and int(val.split('.')[0]) <= 139:
          df[diag].iloc[index] = 'infectious_disease'     # Infectious and parasetic diseases

        elif int(val.split('.')[0]) <= 239:
          df[diag].iloc[index] = 'neoplasms_disease'     # Neoplasms(tumors)

        elif int(val.split('.')[0]) == 250 :
          df[diag].iloc[index] = 'diabets'     # Diabets

        elif int(val.split('.')[0]) <= 289:
          df[diag].iloc[index] = 'blood_disease'     # Diseases of the blood

        elif int(val.split('.')[0]) <= 319:
          df[diag].iloc[index] = 'mental_disorder'   # mental disorders

        elif int(val.split('.')[0]) <= 389:
          df[diag].iloc[index] = 'nervous_sys_disease'     # diseases of the nervous system and sense organs

        elif int(val.split('.')[0]) <= 459:
          df[diag].iloc[index] = 'circulatory_sys_disease'    #  diseases of the circulatory system

        elif int(val.split('.')[0]) <= 519:
          df[diag].iloc[index] = 'respiratory_sys_disease'    # diseases of the respiratory system

        elif int(val.split('.')[0]) <= 579:
          df[diag].iloc[index] = 'digestive_sys_disease'    # diseases of the digestive system

        elif int(val.split('.')[0]) <= 629:
          df[diag].iloc[index] = 'geni_sys_disease'     # diseases of the genitourinary system

        elif int(val.split('.')[0]) <= 679:
          df[diag].iloc[index] = 'complications'    # complications of pregnancy, childbirth, and the puerperium

        elif int(val.split('.')[0]) <= 709:
          df[diag].iloc[index] = 'skin_disease'    # diseases of the skin

        elif int(val.split('.')[0]) <= 739:
          df[diag].iloc[index] = 'muscle disease'     # diseases of the musculoskeletal system and connective tissue

        elif int(val.split('.')[0]) <= 759:
          df[diag].iloc[index] = 'congenital anamoly'    # congenital anomalies

        elif int(val.split('.')[0]) <= 779:
          df[diag].iloc[index] = 'peinatal period'    # certain conditions originating in the perinatal period

        elif int(val.split('.')[0]) <= 799:
          df[diag].iloc[index] = 'ill defined'     # symptoms, signs, and ill-defined conditions

        elif int(val.split('.')[0]) <= 999:
          df[diag].iloc[index] = 'injury and poisoning'     # injury and poisoning

  return df[diag]

In [30]:
df_features['diag_1'] = group_ICD9(df_features,'diag_1').astype('str')
df_features['diag_2'] = group_ICD9(df_features,'diag_2').astype('str')
df_features['diag_3'] = group_ICD9(df_features,'diag_3').astype('str')

/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [31]:
df_test['diag_1'] = group_ICD9(df_test,'diag_1').astype('str')
df_test['diag_2'] = group_ICD9(df_test,'diag_2').astype('str')
df_test['diag_3'] = group_ICD9(df_test,'diag_3').astype('str')

In [32]:
for i in range(len(df_features)):
  if df_features['diag_1'][i] == '?':
    df_features['diag_1'][i] = 'No_diag_1'
  else :
    df_features['diag_1'][i] = df_features['diag_1'][i] + '_diag_1'

  if df_features['diag_2'][i] == '?':
    df_features['diag_2'][i] = 'No_diag_2'
  else :
    df_features['diag_2'][i] = df_features['diag_2'][i] + '_diag_2'

  if df_features['diag_3'][i] == '?':
    df_features['diag_3'][i] = 'No_diag_3'
  else :
    df_features['diag_3'][i] = df_features['diag_3'][i] + '_diag_3'

for i in range(len(df_test)):
  if df_test['diag_1'][i] == '?':
    df_test['diag_1'][i] = 'No_diag_1'
  else :
    df_test['diag_1'][i] = df_test['diag_1'][i] + '_diag_1'

  if df_test['diag_2'][i] == '?':
    df_test['diag_2'][i] = 'No_diag_2'
  else :
    df_test['diag_2'][i] = df_test['diag_2'][i] + '_diag_2'

  if df_test['diag_3'][i] == '?':
    df_test['diag_3'][i] = 'No_diag_3'
  else :
    df_test['diag_3'][i] = df_test['diag_3'][i] + '_diag_3'

<ipython-input-32-ebfca1311b95>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['diag_1'][i] = df_features['diag_1'][i] + '_diag_1'


In [33]:
for i in ['diag_1','diag_2','diag_3']:
  #for the train
  df_diag = pd.get_dummies(df_features[i])
  df_features[df_diag.columns] = df_diag
  df_features.drop(columns=[i],inplace=True)
  #for the test
  df_diag = pd.get_dummies(df_test[i])
  df_test[df_diag.columns] = df_diag
  df_test.drop(columns=[i],inplace=True)

the Race feature

In [34]:
df_race =pd.get_dummies(df_features['race'])
df_features[df_race.columns] = df_race
df_features.drop(columns=['?','race'],inplace=True)

In [35]:
df_race =pd.get_dummies(df_test['race'])
df_test[df_race.columns] = df_race
df_test.drop(columns=['?','race'],inplace=True)

max glu serum

In [36]:
df_serum =pd.get_dummies(df_features['max_glu_serum'])
df_features[df_serum.columns] = df_serum
df_features.drop(columns=['max_glu_serum'],inplace=True)

df_serum_ =pd.get_dummies(df_test['max_glu_serum'])
df_test[df_serum_.columns] = df_serum_
df_test.drop(columns=['max_glu_serum'],inplace=True)

In [37]:
df_serum.columns

Index(['>200', '>300', 'None', 'Norm'], dtype='object')

In [38]:
df_features.rename(columns={'>200':'>200_max_gs','>300':'>300_max_gs','None':'None_max_gs','Norm':'Norm_max_gs'},inplace=True)
df_test.rename(columns={'>200':'>200_max_gs','>300':'>300_max_gs','None':'None_max_gs','Norm':'Norm_max_gs'},inplace=True)

A1Cresult

In [39]:
df_a1 =pd.get_dummies(df_features['A1Cresult'])
df_features[df_a1.columns] = df_a1
df_features.drop(columns=['A1Cresult'],inplace=True)

df_a1_ =pd.get_dummies(df_test['A1Cresult'])
df_test[df_a1_.columns] = df_a1_
df_test.drop(columns=['A1Cresult'],inplace=True)

In [40]:
df_a1.columns

Index(['>7', '>8', 'None', 'Norm'], dtype='object')

In [41]:
df_features.rename(columns={'>7':'>7_a1c','>8':'>8_a1c','None':'None_a1c','Norm':'Norm_a1c'},inplace=True)
df_test.rename(columns={'>7':'>7_a1c','>8':'>8_a1c','None':'None_a1c','Norm':'Norm_a1c'},inplace=True)

extracting features from the medication features

In [42]:
meds = ['metformin','repaglinide','nateglinide','chlorpropamide','glimepiride','acetohexamide','glipizide','glyburide','tolbutamide',
        'pioglitazone','rosiglitazone','acarbose','miglitol','troglitazone','tolazamide','examide',
        'citoglipton','insulin','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone',
        'metformin-rosiglitazone','metformin-pioglitazone']

In [43]:
df_features['glimepiride'].value_counts()

No        67616
Steady     3256
Up          226
Down        138
Name: glimepiride, dtype: int64

In [44]:
df_features['Steady_meds'] = np.zeros(len(df_features))
df_features['Up_meds'] = np.zeros(len(df_features))
df_features['Down_meds'] = np.zeros(len(df_features))

In [45]:
for i in range(len(df_features)):
  df_features['Steady_meds'][i]=(df_features.iloc[i,:] == 'Steady').sum()
  df_features['Up_meds'][i]=(df_features.iloc[i,:] == 'Up').sum()
  df_features['Down_meds'][i]=(df_features.iloc[i,:] == 'Down').sum()

<ipython-input-45-7c0fa6409191>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['Steady_meds'][i]=(df_features.iloc[i,:] == 'Steady').sum()


In [46]:
df_test['Steady_meds'] = np.zeros(len(df_test))
df_test['Up_meds'] = np.zeros(len(df_test))
df_test['Down_meds'] = np.zeros(len(df_test))

In [47]:
for i in range(len(df_test)):
  df_test['Steady_meds'][i]=(df_test.iloc[i,:] == 'Steady').sum()
  df_test['Up_meds'][i]=(df_test.iloc[i,:] == 'Up').sum()
  df_test['Down_meds'][i]=(df_test.iloc[i,:] == 'Down').sum()

<ipython-input-47-e17a0c3ade87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Steady_meds'][i]=(df_test.iloc[i,:] == 'Steady').sum()


label encoding some categorical features

In [88]:
categorical_cols = ['gender','change','diabetesMed']

In [89]:
le = LabelEncoder()
frames = [df_features,df_test]
result = pd.concat(frames)

In [90]:
result = result.reset_index(drop=True)

In [91]:
for col in categorical_cols:

  # fit for train
  le.fit_transform(result[col])

  # transform for test
  df_features[col] = le.transform(df_features[col])
  df_test[col] = le.transform(df_test[col]) 

clustering medications features

In [49]:
df_meds_features = df_features[meds]
df_meds_test = df_test[meds]

In [51]:
!pip install kmodes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/google/colab/_pip.py:83: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.9/dist-packages/kmodes-0.12.2.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


In [68]:
from kmodes.kmodes import KModes

In [70]:
idx = [i for i in range(df_meds_features.shape[1])]

In [79]:
kproto = KModes(n_clusters=6, init='Huang',verbose=2, max_iter=20)
kproto.fit(df_meds_features)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/20, moves: 2586, cost: 61986.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/20, moves: 7356, cost: 40720.0
Run 2, iteration: 2/20, moves: 346, cost: 40720.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/20, moves: 1101, cost: 62193.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/20, moves: 436, cost: 45420.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/20, moves: 2384, cost: 43721.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/20, moves: 824, cost: 45125.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 7, iteration: 1/20, moves: 2005, cost: 45433.0
Init: initializing centroids
Init: 

KModes(init='Huang', max_iter=20, n_clusters=6, verbose=2)

In [80]:
kmodes_classes = kproto.predict(df_meds_features)
kmodes_test_classes = kproto.predict(df_meds_test)

In [81]:
df_features['meds_clusters'] = kmodes_classes
df_test['meds_clusters'] = kmodes_test_classes

In [82]:
pd.concat([df_features['meds_clusters'],labels],axis=1).corr()

,meds_clusters,readmitted
meds_clusters,1.000000,-0.041799
readmitted,-0.041799,1.000000


In [83]:
import joblib

In [85]:
joblib.dump(kproto,'meds_clustering_model.pk1')

['meds_clustering_model.pk1']

In [86]:
df_features.drop(columns = meds,inplace=True)
df_test.drop(columns = meds,inplace=True)

counting total number of patient visists

In [94]:
df_features["patient_visits"]=df_features[['number_outpatient', 'number_emergency', 'number_inpatient']].sum(axis=1)

In [95]:
df_test["patient_visits"]=df_test[['number_outpatient', 'number_emergency', 'number_inpatient']].sum(axis=1)

some other clusterings

In [100]:
df_features.head()

,patient_nbr,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other,patient_sum,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100),E_or_V_diag_1,No_diag_1,blood_disease_diag_1,circulatory_sys_disease_diag_1,complications_diag_1,congenital anamoly_diag_1,diabets_diag_1,digestive_sys_disease_diag_1,geni_sys_disease_diag_1,ill defined_diag_1,infectious_disease_diag_1,injury and poisoning_diag_1,mental_disorder_diag_1,muscle disease_diag_1,neoplasms_disease_diag_1,nervous_sys_disease_diag_1,respiratory_sys_disease_diag_1,skin_disease_diag_1,E_or_V_diag_2,No_diag_2,blood_disease_diag_2,circulatory_sys_disease_diag_2,complications_diag_2,congenital anamoly_diag_2,diabets_diag_2,digestive_sys_disease_diag_2,geni_sys_disease_diag_2,ill defined_diag_2,infectious_disease_diag_2,injury and poisoning_diag_2,mental_disorder_diag_2,muscle disease_diag_2,neoplasms_disease_diag_2,nervous_sys_disease_diag_2,respiratory_sys_disease_diag_2,skin_disease_diag_2,E_or_V_diag_3,No_diag_3,blood_disease_diag_3,circulatory_sys_disease_diag_3,complications_diag_3,congenital anamoly_diag_3,diabets_diag_3,digestive_sys_disease_diag_3,geni_sys_disease_diag_3,ill defined_diag_3,infectious_disease_diag_3,injury and poisoning_diag_3,mental_disorder_diag_3,muscle disease_diag_3,neoplasms_disease_diag_3,nervous_sys_disease_diag_3,respiratory_sys_disease_diag_3,skin_disease_diag_3,AfricanAmerican,Asian,Caucasian,Hispanic,Other,>200_max_gs,>300_max_gs,None_max_gs,Norm_max_gs,>7_a1c,>8_a1c,None_a1c,Norm_a1c,Steady_meds,Up_meds,Down_meds,meds_clusters,patient_visits
0,3,0,3,1,7,5,40,2,25,0,0,0,9,0,1,0,0,0,0,0,0,0,1,0,0,0,0,20.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,2.0,0.0,0.0,0,0
1,1,1,3,5,1,3,22,1,22,0,0,0,8,1,1,0,0,0,0,0,0,0,0,0,0,0,0,15.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1.0,0.0,0.0,0,0
2,1,0,3,3,1,8,33,5,5,0,0,0,9,0,1,0,0,0,0,0,0,0,1,0,0,0,0,20.5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.0,1.0,0.0,3,0
3,1,1,6,1,7,7,52,1,15,0,0,4,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,27.5,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0.0,0.0,0.0,0,4
4,2,1,1,1,6,2,47,2,10,0,0,1,5,1,1,0,0,0,0,0,0,0,0,0,0,1,0,11.5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1.0,0.0,0.0,5,1


In [104]:
procedures_cols = ['time_in_hospital','num_lab_procedures','num_procedures','num_medications','patient_nbr']

In [113]:
from sklearn.cluster import KMeans
kmeanModel = KMeans(n_clusters=4)
kmeanModel.fit(df_features[procedures_cols])

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=4)

In [114]:
patient_procedusres = kmeanModel.predict(df_features[procedures_cols])

In [115]:
df_features['patient_procedures'] = patient_procedusres

In [116]:
patient_procedusres_test = kmeanModel.predict(df_test[procedures_cols])

In [117]:
df_test['patient_procedures'] = patient_procedusres_test

In [118]:
#df_features.drop(columns=['patient_procedures'],inplace=True)

In [119]:
#df_test.drop(columns=['patient_procedures'],inplace=True)

stats

In [123]:
df_test.head()

,patient_nbr,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other,patient_sum,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100),E_or_V_diag_1,No_diag_1,blood_disease_diag_1,circulatory_sys_disease_diag_1,complications_diag_1,congenital anamoly_diag_1,diabets_diag_1,digestive_sys_disease_diag_1,geni_sys_disease_diag_1,ill defined_diag_1,infectious_disease_diag_1,injury and poisoning_diag_1,mental_disorder_diag_1,muscle disease_diag_1,neoplasms_disease_diag_1,nervous_sys_disease_diag_1,respiratory_sys_disease_diag_1,skin_disease_diag_1,E_or_V_diag_2,No_diag_2,blood_disease_diag_2,circulatory_sys_disease_diag_2,complications_diag_2,congenital anamoly_diag_2,diabets_diag_2,digestive_sys_disease_diag_2,geni_sys_disease_diag_2,ill defined_diag_2,infectious_disease_diag_2,injury and poisoning_diag_2,mental_disorder_diag_2,muscle disease_diag_2,neoplasms_disease_diag_2,nervous_sys_disease_diag_2,respiratory_sys_disease_diag_2,skin_disease_diag_2,E_or_V_diag_3,No_diag_3,blood_disease_diag_3,circulatory_sys_disease_diag_3,complications_diag_3,congenital anamoly_diag_3,diabets_diag_3,digestive_sys_disease_diag_3,geni_sys_disease_diag_3,ill defined_diag_3,infectious_disease_diag_3,injury and poisoning_diag_3,mental_disorder_diag_3,muscle disease_diag_3,neoplasms_disease_diag_3,nervous_sys_disease_diag_3,respiratory_sys_disease_diag_3,skin_disease_diag_3,AfricanAmerican,Asian,Caucasian,Hispanic,Other,>200_max_gs,>300_max_gs,None_max_gs,Norm_max_gs,>7_a1c,>8_a1c,None_a1c,Norm_a1c,Steady_meds,Up_meds,Down_meds,meds_clusters,patient_visit,patient_procedures
0,1,0,2,22,1,4,46,1,26,0,0,0,9,1,1,0,0,0,0,0,0,0,1,0,0,0,0,32.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1.0,0.0,0.0,5,0,3
1,1,1,2,13,1,3,42,0,12,0,0,0,9,0,1,0,0,0,0,0,0,0,1,0,0,0,0,21.5,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,2.0,0.0,0.0,5,0,0
2,1,0,2,18,4,4,71,3,20,0,0,0,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,31.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0.0,0.0,0.0,0,0,2
3,1,0,1,22,7,7,79,1,13,0,0,0,9,1,1,0,0,0,0,0,0,0,0,1,0,0,0,41.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1.0,0.0,0.0,5,0,2
4,2,1,2,22,1,7,33,2,37,0,0,4,9,1,1,0,0,0,0,0,0,0,0,0,0,0,0,36.5,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1.0,0.0,0.0,0,4,0


In [120]:
d =pd.concat((df_features,labels),axis=1)
d.corr().tail()

,patient_nbr,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other,patient_sum,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100),E_or_V_diag_1,No_diag_1,blood_disease_diag_1,circulatory_sys_disease_diag_1,complications_diag_1,congenital anamoly_diag_1,diabets_diag_1,digestive_sys_disease_diag_1,geni_sys_disease_diag_1,ill defined_diag_1,infectious_disease_diag_1,injury and poisoning_diag_1,mental_disorder_diag_1,muscle disease_diag_1,neoplasms_disease_diag_1,nervous_sys_disease_diag_1,respiratory_sys_disease_diag_1,skin_disease_diag_1,E_or_V_diag_2,No_diag_2,blood_disease_diag_2,circulatory_sys_disease_diag_2,complications_diag_2,congenital anamoly_diag_2,diabets_diag_2,digestive_sys_disease_diag_2,geni_sys_disease_diag_2,ill defined_diag_2,infectious_disease_diag_2,injury and poisoning_diag_2,mental_disorder_diag_2,muscle disease_diag_2,neoplasms_disease_diag_2,nervous_sys_disease_diag_2,respiratory_sys_disease_diag_2,skin_disease_diag_2,E_or_V_diag_3,No_diag_3,blood_disease_diag_3,circulatory_sys_disease_diag_3,complications_diag_3,congenital anamoly_diag_3,diabets_diag_3,digestive_sys_disease_diag_3,geni_sys_disease_diag_3,ill defined_diag_3,infectious_disease_diag_3,injury and poisoning_diag_3,mental_disorder_diag_3,muscle disease_diag_3,neoplasms_disease_diag_3,nervous_sys_disease_diag_3,respiratory_sys_disease_diag_3,skin_disease_diag_3,AfricanAmerican,Asian,Caucasian,Hispanic,Other,>200_max_gs,>300_max_gs,None_max_gs,Norm_max_gs,>7_a1c,>8_a1c,None_a1c,Norm_a1c,Steady_meds,Up_meds,Down_meds,meds_clusters,patient_visits,patient_procedures,readmitted
Down_meds,0.067842,0.000638,-0.001049,-0.001070,0.024893,0.083241,0.068500,0.005930,0.146433,0.010919,0.027019,0.050706,0.051708,-0.427778,0.215225,-0.009819,-0.001484,-0.039593,0.002868,0.009080,0.003453,-0.030496,0.026176,0.048266,0.066379,-0.002655,0.029346,0.051927,-0.004357,0.008982,0.024303,0.014954,0.022525,0.010772,0.000637,-0.016313,-0.022727,-0.010720,-0.000581,0.000947,-0.003651,-0.013499,0.009230,-0.003632,0.077990,-0.029247,-0.003121,-0.031960,0.008739,-0.004664,-0.006520,-0.016560,-0.010429,-0.006398,0.023140,0.012132,-0.000755,0.003723,-0.000481,-0.028828,0.006850,-0.005846,0.022750,-0.009311,0.012661,-0.016097,0.013660,0.000525,0.000203,0.005185,-0.014809,-0.003449,0.006117,0.030586,-0.007897,0.005679,0.007234,-0.016973,0.006705,-0.002825,-0.005289,-0.003729,0.021584,-0.015599,0.006595,0.004037,-0.000757,-0.000464,-0.004735,0.004665,0.005885,0.027207,0.000980,-0.001594,-0.002389,0.002018,0.019845,0.023531,0.043574,-0.026847,-0.010340,0.004840,0.073651,-0.056764,0.000738,-0.164846,-0.098975,1.000000,0.240588,0.045081,0.040365,-0.049448
meds_clusters,0.054387,0.001713,-0.030748,-0.043866,0.018194,0.064093,0.072897,0.019764,0.136960,0.000011,0.029022,0.068267,0.064728,-0.278082,0.533148,-0.014398,-0.003579,-0.070475,-0.001012,0.024828,-0.009622,-0.054609,0.055488,0.047432,0.017637,0.010808,0.027665,0.007038,0.035066,0.060359,0.047183,0.019222,0.008929,0.008986,-0.005684,-0.026517,-0.017491,-0.009114,-0.021015,-0.002937,0.002452,-0.016725,0.020422,-0.002409,0.109045,0.004513,0.006871,-0.051459,0.029445,-0.003974,-0.036318,-0.056894,-0.016510,-0.014792,0.024543,-0.003099,-0.012589,0.026033,0.018574,-0.030780,0.009429,-0.000856,0.014034,0.001533,0.032074,-0.020348,0.009359,-0.009414,-0.015041,-0.013208,-0.014185,-0.012213,0.007995,0.024519,-0.011077,0.027316,0.019837,-0.023858,0.008922,0.006316,-0.020878,-0.000757,0.029536,-0.004317,0.011790,0.004865,-0.008004,-0.009104,-0.006279,-0.004006,0.014730,0.018911,0.048001,-0.010830,-0.051838,-0.003577,0.011072,0.026628,0.028674,-0.003109,-0.035725,0.000975,0.079671,-0.058716,-0.000112,0.176156,0.096748,0.240588,1.000000,0.049626,0.039454,-0.041799
patient_visits,0.488710,-0.0

Saving the processed data

In [129]:
df_features.to_csv('df_features.csv',index=False)
df_test.to_csv('df_test.csv',index=False)

explorations

In [150]:
df_features.head()

,patient_nbr,gender,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,change,diabetesMed,>125,[0-50),[50-125),BC,CM,CP,HM,MC,MD,SP,UN,other,patient_sum,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100),E_or_V_diag_1,No_diag_1,blood_disease_diag_1,circulatory_sys_disease_diag_1,complications_diag_1,congenital anamoly_diag_1,diabets_diag_1,digestive_sys_disease_diag_1,geni_sys_disease_diag_1,ill defined_diag_1,infectious_disease_diag_1,injury and poisoning_diag_1,mental_disorder_diag_1,muscle disease_diag_1,neoplasms_disease_diag_1,nervous_sys_disease_diag_1,respiratory_sys_disease_diag_1,skin_disease_diag_1,E_or_V_diag_2,No_diag_2,blood_disease_diag_2,circulatory_sys_disease_diag_2,complications_diag_2,congenital anamoly_diag_2,diabets_diag_2,digestive_sys_disease_diag_2,geni_sys_disease_diag_2,ill defined_diag_2,infectious_disease_diag_2,injury and poisoning_diag_2,mental_disorder_diag_2,muscle disease_diag_2,neoplasms_disease_diag_2,nervous_sys_disease_diag_2,respiratory_sys_disease_diag_2,skin_disease_diag_2,E_or_V_diag_3,No_diag_3,blood_disease_diag_3,circulatory_sys_disease_diag_3,complications_diag_3,congenital anamoly_diag_3,diabets_diag_3,digestive_sys_disease_diag_3,geni_sys_disease_diag_3,ill defined_diag_3,infectious_disease_diag_3,injury and poisoning_diag_3,mental_disorder_diag_3,muscle disease_diag_3,neoplasms_disease_diag_3,nervous_sys_disease_diag_3,respiratory_sys_disease_diag_3,skin_disease_diag_3,AfricanAmerican,Asian,Caucasian,Hispanic,Other,>200_max_gs,>300_max_gs,None_max_gs,Norm_max_gs,>7_a1c,>8_a1c,None_a1c,Norm_a1c,Steady_meds,Up_meds,Down_meds,meds_clusters,patient_visits,patient_procedures
0,3,0,3,1,7,5,40,2,25,0,0,0,9,0,1,0,0,0,0,0,0,0,1,0,0,0,0,20.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,2.0,0.0,0.0,0,0,0
1,1,1,3,5,1,3,22,1,22,0,0,0,8,1,1,0,0,0,0,0,0,0,0,0,0,0,0,15.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1.0,0.0,0.0,0,0,1
2,1,0,3,3,1,8,33,5,5,0,0,0,9,0,1,0,0,0,0,0,0,0,1,0,0,0,0,20.5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0.0,1.0,0.0,3,0,0
3,1,1,6,1,7,7,52,1,15,0,0,4,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,27.5,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0.0,0.0,0.0,0,4,3
4,2,1,1,1,6,2,47,2,10,0,0,1,5,1,1,0,0,0,0,0,0,0,0,0,0,1,0,11.5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1.0,0.0,0.0,5,1,3


In [128]:
for col in df_features.columns:
    print(col,"----->", df_features[col].var())
    print(col,"----->", df_test[col].var(),'\n')

patient_nbr -----> 3.0012350025365873
patient_nbr -----> 0.8600807872647483 

gender -----> 0.24861724999607154
gender -----> 0.24873751833819882 

admission_type_id -----> 2.0932773936746423
admission_type_id -----> 2.0797153752538073 

discharge_disposition_id -----> 27.745023729704208
discharge_disposition_id -----> 28.1960902188151 

admission_source_id -----> 16.617560473182397
admission_source_id -----> 16.28144676451338 

time_in_hospital -----> 8.898011286030142
time_in_hospital -----> 8.94113313210744 

num_lab_procedures -----> 386.3235031495125
num_lab_procedures -----> 388.8595012306041 

num_procedures -----> 2.9096321779678616
num_procedures -----> 2.910166372231578 

num_medications -----> 65.65844569650167
num_medications -----> 66.99005931299615 

number_outpatient -----> 1.5480222057342807
number_outpatient -----> 1.7411794868983315 

number_emergency -----> 0.8095780054992886
number_emergency -----> 0.9969057187327761 

number_inpatient -----> 1.5678170416932362
numb

remove low variance features

In [ ]:
#from sklearn.feature_selection import VarianceThreshold

#var_thr = VarianceThreshold(threshold = 0.1) #Removing both constant and quasi-constant
#var_thr.fit(df_features)

#df_var = var_thr.transform(df_features)
#df_var

array([[ 3. ,  3. ,  0. , ...,  1. , 20. ,  6. ],
       [ 1. ,  3. ,  1. , ...,  1. , 15. ,  7. ],
       [ 1. ,  3. ,  0. , ...,  1. , 20.5,  3. ],
       ...,
       [ 1. ,  3. ,  1. , ...,  1. , 14. ,  1. ],
       [ 1. ,  3. ,  0. , ...,  0. , 11. ,  0. ],
       [ 1. ,  3. ,  1. , ...,  1. , 12.5,  6. ]])

In [ ]:
#df_test.drop(columns =['readmitted'],inplace=True)
#X_test =  var_thr.transform(df_test) 
#X_test.shape

(30530, 24)

apply test train split to get validation data

In [130]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df_features,labels, test_size=0.1, random_state=42,
                                                  stratify=labels 
                                                  )

scaling the data

In [131]:
from sklearn.preprocessing import StandardScaler , MinMaxScaler , RobustScaler
scaler= StandardScaler()
scaler.fit_transform(X_train)

array([[-0.50220635, -0.92540897, -0.7073884 , ..., -0.98070267,
        -0.53265113, -1.17949588],
       [-0.50220635, -0.92540897,  2.05667645, ..., -0.02718721,
        -0.53265113,  0.4052277 ],
       [-0.50220635, -0.92540897, -0.7073884 , ..., -0.98070267,
        -0.53265113, -1.17949588],
       ...,
       [-0.50220635, -0.92540897,  0.67464403, ...,  1.40308599,
        -0.53265113, -1.17949588],
       [-0.50220635, -0.92540897,  0.67464403, ...,  1.40308599,
        -0.53265113, -1.17949588],
       [ 2.97621781, -0.92540897, -0.01637219, ..., -0.98070267,
         1.70342335,  1.19758949]])

In [132]:
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [133]:
X_test = scaler.transform(df_test)

training the model

In [146]:
from sklearn.ensemble import HistGradientBoostingClassifier
#from imblearn.over_sampling import SMOTE
#oversample = SMOTE(sampling_strategy = {0:10000,1:25000,2:40000})
#over_X, over_y = oversample.fit_resample(X_train, y_train)
hclf = HistGradientBoostingClassifier(max_iter=2000,max_depth=9,random_state=42
                                      )
hclf.fit(X_train,y_train)

HistGradientBoostingClassifier(max_depth=9, max_iter=2000, random_state=42)

In [147]:
hclf.score(X_train,y_train)

0.7118324182680309

In [148]:
hclf.score(X_val,y_val)

0.6882369455362156

In [137]:
y_val_preds = hclf.predict(X_val)
np.unique(y_val_preds,return_counts=True)

(array([0, 1, 2]), array([  80, 2290, 4754]))

In [138]:
np.unique(y_val,return_counts=True)

(array([0, 1, 2]), array([ 795, 2488, 3841]))

In [ ]:
!pip install catboost

In [151]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(
    iterations=1000, 
    learning_rate=0.1, 
)

cat.fit(X_train, y_train, 
        #cat_features=cat_features, 
        eval_set=(X_val, y_val), 
        verbose=False
)

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [152]:
cat.score(X_val,y_val)

0.6904828747894441

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=500,random_state=20
                                 ,max_depth = 5
                                 )
clf.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=5, n_estimators=500, random_state=20)

In [ ]:
clf.score(X_train,y_train)

0.754558619262937

In [ ]:
clf.score(X_val,y_val)

0.6855699045480067

In [ ]:
y_val_preds = clf.predict(X_val)
np.unique(y_val_preds,return_counts=True)

(array([0, 1, 2]), array([  53, 1088, 2421]))

In [ ]:
np.unique(y_val,return_counts=True)

(array([0, 1, 2]), array([ 398, 1244, 1920]))

In [ ]:
import xgboost as xgb
xg = xgb.XGBClassifier(n_estimators = 500 , random_state=20 , )
xg.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
xg.score(X_val,y_val)

0.6305446378439079

In [ ]:
y_val_preds = xg.predict(X_val)
np.unique(y_val_preds,return_counts=True)

In [ ]:
np.unique(y_val,return_counts=True)

predictions and submission

In [153]:
df_submit= pd.read_csv('/content/drive/MyDrive/kaggle_readmission_comp/test.csv')

In [154]:
y_pred = cat.predict(X_test)
predictions = labels_encoder.inverse_transform(y_pred)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [159]:
df_test['readmitted'] = predictions
df_test['encounter_id'] = df_submit['encounter_id']
df_test[['encounter_id', 'readmitted']].to_csv('submission.csv', index=False)

In [161]:
df_test['readmitted'].value_counts()

NO     24095
>30     6245
<30      190
Name: readmitted, dtype: int64